In [1]:
import logging
import os
import socket
import sys
import time

import numpy as np
import pickle as pk
import tenseal as ts

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, accuracy_score
                           
os.chdir("/home/apignet/homomorphic-encryption/ckks_titanic/")
from src.features import build_features
from models import encrypted_LR
from models import unencrypted_LR
from models.Actors import *

In [2]:
%load_ext memory_profiler

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
1

1

# definition of parameters

### Paths

In [4]:
WORKING_DIR = os.getcwd()

LOG_PATH = WORKING_DIR+"/reports/log/"
LOG_FILENAME = "Alice"

### Paths

In [5]:
fileHandler = logging.FileHandler("{0}/{1}.log".format(LOG_PATH, LOG_FILENAME))
streamHandler = logging.StreamHandler(sys.stdout)
logging.basicConfig(format="%(asctime)s  [%(levelname)-8.8s]  %(message)s", 
                    datefmt='%m/%d/%Y %I:%M:%S %p', 
                    level = logging.INFO, 
                    handlers=[fileHandler, streamHandler]
                   )

### Training variables

In [6]:
EPOCH = 40
LEARNING_RATE = 0.9
MOMENTUM_RATE = 0.6
REGULARIZATION_RATE = 0.5
VERBOSE = 4
SAVE_WEIGHT = 4
N_JOBS = 8

### Socket Variables

In [7]:
HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65432        # The port used by the server
PACKETS_SIZE = 16384
SENTINEL = b'BREAK'

# Esthablishing the communication with Bob


In [8]:
BOB = Actor(HOST, PORT, PACKETS_SIZE, SENTINEL)

08/06/2020 12:08:38 PM  [INFO    ]  Nobody is listening, binding a server...
08/06/2020 12:09:20 PM  [INFO    ]  Connected by 127.0.0.1:38318


## Receiving the public context

In [9]:
logging.info('Waiting for serialized context')

b_context = BOB.reception()
logging.info('Context received. Starting deserialization.')
context = ts.context_from(b_context)
logging.info('Context successfully deserialized')

08/06/2020 12:09:20 PM  [INFO    ]  Waiting for serialized context
08/06/2020 12:09:21 PM  [INFO    ]  Context received. Strating deserialization.


NameError: name 'b_data' is not defined

In [11]:
context = ts.context_from(b_context)
context

## Receiving serialized data

In [14]:
b_len_X = BOB.reception()
len_X = pk.loads(b_len_X)
b_Y = [None for _ in range(len_X)]
b_X = [None for _ in range(len_X)]

In [15]:
while True:
    b_data = BOB.reception()
    if b_data==b'DONE': break
    data = pk.loads(b_data)
    b_X[data[0]]=data[1]
    b_Y[data[0]]=data[2]
    
    if data[0] == len(b_X) // 4:
        logging.info("25 % ...")
    elif data[0] == len(b_X) // 2 :
        logging.info("50 % ...")
    elif data[0] == 3* len(b_X)//4:
        logging.info("75% ...")
    
assert not (None in b_X): 'Missing value: ' + str([idx for idx,e in enumerate(b_X) if e is None])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Initialize the weight

The weights have to be crypted 

In [13]:
%%memit
unencrypted_weight = np.random.normal(loc=0,
                                      scale=0.2, size =(train_features.to_numpy().shape[1]))
logging.info(unencrypted_weight)
weight = ts.ckks_vector(context, unencrypted_weight.tolist())
unencrypted_bias = np.random.random((1))
bias = ts.ckks_vector(context, unencrypted_bias.tolist())

08/04/2020 01:45:36 PM  [INFO    ]  [ 0.20792981 -0.20951224 -0.09917668 -0.12221482  0.00526441 -0.1687017
 -0.26214529  0.07401318 -0.07199249  0.35331028  0.10488685  0.03633133
  0.22256725 -0.12558542 -0.17193498  0.1115622  -0.03979612 -0.13965522
 -0.08638655  0.23146916  0.06489733 -0.0298103  -0.08953358  0.10140427
  0.0674875   0.02312168 -0.00952971  0.12402542 -0.24957804 -0.20083211
 -0.33674684 -0.02919112]
08/04/2020 01:45:36 PM  [INFO    ]  [ 0.35151082 -0.11324081  0.10799244  0.1390745   0.0174161  -0.00993639
  0.02737775  0.03630417  0.24638261 -0.22878206 -0.3577683   0.23218647
 -0.27641665 -0.37296542 -0.04908337 -0.35810506  0.37795586  0.14762392
 -0.26030713 -0.06015141  0.00648724 -0.13970583 -0.09861461  0.20008066
 -0.05734391 -0.04100004 -0.1493433   0.14257616 -0.17017428  0.20667327
  0.09623054  0.22968221]
peak memory: 3322.08 MiB, increment: 6.03 MiB


# Training the crypted models

### Initialization of the model...

In [20]:
logging.info("Model initialization")
model = encrypted_LR.LogisticRegressionHE(init_weight=weight,
                                          init_bias=bias,
                                          bob=BOB,
                                          context=context,
                                          verbose=VERBOSE,
                                          save_weight=SAVE_WEIGHT,
                                          max_epoch=EPOCH,
                                          learning_rate=LEARNING_RATE,
                                          momentum_rate=MOMENTUM_RATE,
                                          reg_para=REGULARIZATION_RATE,
                                          n_jobs = N_JOBS)

08/04/2020 01:45:38 PM  [INFO    ]  Model initialization


### Warning Bob that Alice is starting the training
Indeed Alice needs Bob to be ready to refresh the weights

In [21]:
logging.info("Training starting")
timer=time.time()
model.fit(encrypted_X, encrypted_Y)
logging.info("Training done. " + str(round(time.time() - timer, 0)) + " seconds")

08/04/2020 01:46:27 PM  [INFO    ]  Training starting
08/04/2020 01:46:27 PM  [INFO    ]  Starting serialization of data
08/04/2020 01:50:21 PM  [INFO    ]  Data serialization done in 234.05052828788757 seconds
08/04/2020 01:50:21 PM  [INFO    ]  Initialization of 8 workers
08/04/2020 01:50:29 PM  [INFO    ]  Initialization done for process Process-17. Len of data : 69
08/04/2020 01:50:29 PM  [INFO    ]  Initialization done for process Process-18. Len of data : 69
08/04/2020 01:50:30 PM  [INFO    ]  Initialization done for process Process-20. Len of data : 68
08/04/2020 01:50:30 PM  [INFO    ]  Initialization done for process Process-19. Len of data : 68
08/04/2020 01:50:30 PM  [INFO    ]  Initialization done for process Process-21. Len of data : 68
08/04/2020 01:50:30 PM  [INFO    ]  Initialization done for process Process-22. Len of data : 68
08/04/2020 01:50:30 PM  [INFO    ]  Initialization done for process Process-23. Len of data : 68
08/04/2020 01:50:31 PM  [INFO    ]  Initializa